In [188]:
import pandas as pd
import re
import math
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import sentencepiece as spm

from tqdm import tqdm
import random

file = "amazon_review.csv"

data = pd.read_csv(file).dropna(ignore_index=True)
data['overall'] = data["overall"] - 1

vocab_size = 16000
seq_len = 512

spm.SentencePieceTrainer.train(
    input='amazon_reviews.txt',
    model_prefix='amazon_reviews',
    vocab_size=vocab_size,
    model_type='unigram',
    character_coverage=1.0
)

tok = spm.SentencePieceProcessor(model_file='amazon_reviews.model')

filter_ = 1

ls  = [len(tok.encode(i, out_type=int)) for i in data["reviewText"]]

data["lengths"] = ls

data_trunc = data[data["lengths"]<=80]

v = data_trunc["overall"].value_counts()

balanced_data = (
    data_trunc.groupby("overall")
      .sample(n=min(v), random_state=42)
      .reset_index(drop=True)
)

len(balanced_data)

df_shuffled = balanced_data.sample(frac=filter_, random_state=42).reset_index(drop=True)

# Split the shuffled DataFrame
train_size = 0.8
train_df = df_shuffled.sample(frac=train_size, random_state=42).reset_index(drop=True)
test_df = df_shuffled.drop(train_df.index).reset_index(drop=True)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: amazon_reviews.txt
  input_format: 
  model_prefix: amazon_reviews
  model_type: UNIGRAM
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0

In [189]:
print("len train df: ", len(train_df))
print("len test df: ", len(test_df))

len train df:  15396
len test df:  3849


In [190]:
# get one input output pair with special tokens for later concatenation with other pairs for one sequence under max sequence length
def getEncodingOpen(df, i):
    reviewtext = "Review: "+ df["reviewText"].iloc[i]
    rating = "Rating: " 
    #row = [16000] + tok.encode(reviewText, out_type=int) + [16001] + tok.encode([int(df["overall"].iloc[i])], out_type = int)
    row = [16000] + tok.encode(reviewtext, out_type = int) + [16002] + tok.encode(rating, out_type = int)
    correct_output_rating = int(df["overall"].iloc[i])
    row = torch.LongTensor(row)
    correct_output_rating = torch.LongTensor([correct_output_rating])
    return row, correct_output_rating

In [191]:
# get one input output pair with special tokens for later concatenation with other pairs for one sequence under max sequence length
def getEncoding(df, i):
    reviewtext = "Review: "+ df["reviewText"].iloc[i]
    rating = "Rating: " + str(int(df["overall"].iloc[i])+16003)
    #row = [16000] + tok.encode(reviewText, out_type=int) + [16001] + tok.encode([int(df["overall"].iloc[i])], out_type = int)
    row = [16000] + tok.encode(reviewtext, out_type = int) + [16002] + tok.encode(rating, out_type = int) + [16001]
    return row

def getShiftSeq(df_t, max_seq=1024):
    seqs_x = []
    seqs_y = []
    seqs = []
    c = []
    for i in range(len(df_t)):
        row = getEncoding(train_df, i)
        if len(c) + len(row) > max_seq +1:
            seqs_x.append(c[:-1])
            seqs_y.append(c[1:])
            seqs.append(c)
            c = []
        c.extend(row)
    return seqs_x, seqs_y, seqs

train_seqs_x, train_seqs_y, train_seqs = getShiftSeq(train_df, max_seq=seq_len)
test_seqs_x, test_seqs_y, test_seqs = getShiftSeq(test_df, max_seq=seq_len)

In [192]:
class TokenDatasetB(Dataset):
    def __init__(self, seqs_x, seqs_y):
        self.seqs_x = seqs_x
        self.seqs_y = seqs_y
    
    def __len__(self):
        return len(self.seqs_x)
    
    def __getitem__(self, idx):
        return torch.LongTensor(self.seqs_x[idx]), torch.LongTensor(self.seqs_y[idx])

# Padding collate function for variable length sequences
def collate_fnB(batch):
    seqs_x, seqs_y = zip(*batch)
    lens = [len(s) for s in seqs_x]
    max_len = max(lens)
    
    # Pad sequences
    padded_x = torch.zeros(len(seqs_x), max_len, dtype=torch.long)
    padded_y = torch.zeros(len(seqs_y), max_len, dtype=torch.long)
    
    for i, (x, y) in enumerate(zip(seqs_x, seqs_y)):
        padded_x[i, :len(x)] = x
        padded_y[i, :len(y)] = y
    
    return padded_x, padded_y, torch.LongTensor(lens)

dataset = TokenDatasetB(train_seqs_x, train_seqs_y)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fnB)

In [193]:
for batch_idx, (x, y, lengths) in enumerate(train_loader):
    print("bi")
    print(batch_idx)
    print("x")
    print(x)
    print(x.shape)
    print("y")
    print(y)
    print(y.shape)
    print("lengths")
    print(lengths.shape)
    break

lengths[:10]

bi
0
x
tensor([[16000,  7421,   292,   139,  1183,     6,    36,   297,    61,    23,
           994,     3, 12660,    26,   231,   158,   245,    30,    79,     6,
             4,    41,     8,   381,  1164,     7,   137,    31,    16,     8,
           381,     3,  1716,    35,   495,    67,  6214, 16002, 13254,   292,
          4037, 12507, 16001, 16000,  7421,   292,    45,    19,    49,     3,
          1051,    47,     7,    19,  1213,  1125,    37,  2363,  2290,  1741,
             3, 16002, 13254,   292,  1736, 12473, 16001, 16000,  7421,   292,
           949,    14,    10,    15,     5,   390,     6,    23,    24,   301,
             9,   257,     9,  1165,     8,    74,   551,     3,    39,   715,
           143,   277,  1776,   172,   807,   150,   152,   114,     8,   774,
             3, 16002, 13254,   292,  1736,  2301,  1974, 16001, 16000,  7421,
           292,   815,     8,  1738,     7,    14,    10,    15,  8966,  4124,
            53,   117,     3,    39,   971,  

tensor([506])

In [194]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")
    device = 'cuda'
else:
    print("CUDA is not available. PyTorch will use the CPU.")
    device = 'cpu'

CUDA is available!
CUDA device count: 1
Current CUDA device name: NVIDIA GeForce RTX 4080 SUPER


In [195]:
class GPTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super().__init__()

        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim, num_heads, dropout=dropout, batch_first=True
        )

        self.ln2 = nn.LayerNorm(embed_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x, causal_mask, padding_mask):
        # Self-attention (GPT-style)
        h = self.ln1(x)
        attn_out, _ = self.attn(
            h, h, h,
            attn_mask=causal_mask,
            key_padding_mask=padding_mask,
            need_weights=False
        )
        x = x + attn_out

        # Feedforward
        h = self.ln2(x)
        ff_out = self.mlp(h)
        x = x + ff_out

        return x
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, max_len=512,
                 embed_dim=512, num_heads=4,
                 num_layers=4, mlp_dim=1024, dropout=0.2):
        super().__init__()

        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos = nn.Embedding(max_len, embed_dim)

        self.layers = nn.ModuleList([
            GPTBlock(embed_dim, num_heads, mlp_dim, dropout)
            for _ in range(num_layers)
        ])

        self.ln_final = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size, bias=False)
        self.head.weight = self.embed.weight  # weight tying

    def causal_mask(self, T, device):
        mask = torch.triu(torch.ones(T, T, device=device), 1)
        return mask.masked_fill(mask == 1, float('-inf'))

    def forward(self, x, lengths=None):
        B, T = x.shape
        device = x.device

        tok = self.embed(x)
        pos = self.pos(torch.arange(T, device=device)[None, :])
        h = tok + pos

        causal = self.causal_mask(T, device)     # (T, T)
        pad_mask = (x == 0)                      # (B, T)

        for layer in self.layers:
            h = layer(h, causal, pad_mask)

        h = self.ln_final(h)
        return self.head(h)                      # (B, T, V)


In [196]:
# Training function
def train_model(model, train_loader, epochs=10, lr=1e-4, device='cuda'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    track_loss = []
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loader = tqdm(train_loader)
        
        for x, y, lengths in loader:
            x, y = x.to(device), y.to(device)
            lengths = lengths.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            logits = model(x, lengths)
            
            # Reshape for loss calculation
            logits = logits.view(-1, logits.size(-1))
            y = y.view(-1)
            
            loss = criterion(logits, y)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            track_loss.append(loss.item())
            avg_loss = sum(track_loss[-10:]) / 10
            loader.set_postfix(loss=avg_loss)
            del logits
            torch.cuda.empty_cache()
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}')
    
    return model




In [197]:
dataset = TokenDatasetB(train_seqs_x, train_seqs_y)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fnB)


# Initialize model (set vocab_size to your tokenizer's vocab size + special tokens)
vocab_size = 16000+8  # Adjust based on your tokenizer
model = DecoderOnlyTransformer(vocab_size=vocab_size,  num_layers=12)

In [198]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 33692672


In [199]:
# Train
model = train_model(model, train_loader, epochs=20, lr=1e-4)


  0%|                                                   | 0/282 [00:00<?, ?it/s]/home/rohan/Desktop/berkeley/cs182/cs182/lib/python3.10/site-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████████████████████████| 282/282 [00:13<00:00, 20.60it/s, loss=11.1]


Epoch 1/20, Average Loss: 23.9518


100%|██████████████████████████████| 282/282 [00:13<00:00, 20.47it/s, loss=7.31]


Epoch 2/20, Average Loss: 8.8929


100%|██████████████████████████████| 282/282 [00:13<00:00, 20.50it/s, loss=6.04]


...

Epoch 18/20, Average Loss: 3.7338


100%|██████████████████████████████| 282/282 [00:13<00:00, 20.30it/s, loss=3.66]


Epoch 19/20, Average Loss: 3.6852


100%|██████████████████████████████| 282/282 [00:13<00:00, 20.35it/s, loss=3.63]

Epoch 20/20, Average Loss: 3.6347


In [200]:
k = [345, 23, 44, 99]
kt = torch.tensor(k)

In [201]:
seq_len

512

In [202]:
anno, score = getEncodingOpen(test_df, random.randint(0,len(test_df)))

In [203]:
annot = torch.tensor(anno)

/tmp/ipykernel_19232/4078374235.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  annot = torch.tensor(anno)


In [204]:
lens = len(anno)
anno_len = torch.tensor([lens])

# Pad sequences
padded_x = torch.zeros(1, 512, dtype=torch.long)
padded_x[0,:lens] = torch.tensor(anno)

padded_x = padded_x.to(device)
anno_len = anno_len.to(device)

/tmp/ipykernel_19232/652292005.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_x[0,:lens] = torch.tensor(anno)


In [205]:
print(lens)

10


In [206]:
out = model(padded_x, anno_len)

In [207]:
out.shape

torch.Size([1, 512, 16008])

In [208]:
tokenout = torch.argmax(out,dim=2)[0]

In [209]:
score

tensor([1])

In [210]:
tokenout.shape

torch.Size([512])

In [211]:

tok.decode([min(15999,to) for to in tokenout.tolist()])

'Review: I loves small.{ Rating: 16 the 160{.. a the{ it the the 2t\' 160{ to{ the{ the to to{ with size the{ times,{{. it a 160 to{ a.{ the{ this small the{{{ these{ large{.\' waist{{ the{{ a{ large{ 160{ing{.e{ the,, 160 160 them 160{ the{ing, a{ the{, 160 them the them, of shirts{ the{ thant," the{ not to my" to 160 to 160 to the{ not, thed years 160 at is{{ pair than sizes to the toing the them is size is them{ than with, them than tot{ the them the". with after the{ them my 160 than to.{ 160 to"{ to. to.. to, 160 them bigger{,,\'ing the 160 a{{{{ 160 the on{ the the. them.\'{.{ to on the{ to. 160 160.,{{ because the the to 160 160 the{ them pants{ the the.. on than{ a the the{{ at is the{ them is, the not,{ the.{{ smalled size 160 the 160,{ always small use after{ is/ 160{ what. on the my{" them the size to{{ is",/".ing at{ and\'?{.\'/,. size the{,{ out 160{{ 160{ 160 large the than small the than: them{ have 160 the{ 160 seem 160 love{\' to the{. 160 and the the. 160{ it I theing

In [212]:
class ICLDataset(Dataset):
    def __init__(self, df, shots, seq_len):
        self.df = df
        self.shots = shots
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.df)

    def getpre(self, idx, l):
        for _ in range(10):
            ixs = []
            xs = []
            for j in range(self.shots):
                sel = idx
                while(sel==idx):
                    sel = random.randint(0, self.__len__() -1)
                xs.extend(getEncoding(self.df, sel))
            if(len(xs) + l <= seq_len):
                return torch.LongTensor(xs)
        raise ValueError(f"can't fit {self.shots} examples in context")
                
            
    
    def __getitem__(self, idx):
        x, y = getEncodingOpen(self.df, idx)
        l = len(x)
        pre = self.getpre(idx, l)
        icl_x = torch.cat((pre,x))
        return icl_x, y

# Padding collate function for variable length sequences
def collate_fn_icl(batch):
    seqs_x, y = zip(*batch)
    lens = [len(s) for s in seqs_x]
    max_len = max(lens)
    
    # Pad sequences
    padded_x = torch.zeros(len(seqs_x), max_len, dtype=torch.long)
    
    for i, x in enumerate(seqs_x):
        padded_x[i, :len(x)] = x
    
    return padded_x, torch.LongTensor(y), torch.LongTensor(lens)



In [213]:
icl_dataset = ICLDataset(train_df, 3, seq_len)
icl_loader = DataLoader(icl_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn_icl)

for batch_idx, (x, y, lengths) in enumerate(icl_loader):
    print("bi")
    print(batch_idx)
    print("x")
    print(x)
    print(x.shape)
    print("y")
    print(y)
    print(y.shape)
    print("lengths")
    print(lengths.shape)
    break

In [214]:
def israting(s):
    bnk = ["0","1","2","3","4"]
    return s in bnk
def isnum(s):
    bnk = [str(n) for n in range(0,20)]
    return s in bnk

def check_token_list(token_list):
    isratings = 0
    isnums = 0
    for token in token_list:
        isratings += 1 if israting(token) else 0
        isnums += 1 if isnum(token) else 0
    return isratings, isnums

In [215]:
tok.decode([15999])

'{'

In [216]:
tok.decode([])

''

In [217]:
d = {16000: "<BOS>", 16001: "<EOS>",16002: "<SEP>",16003: "<0>",16004: "<1>",16005: "<2>",16006: "<3>",16007: "<4>"}

In [218]:
def decode_seq(seq):
    outp = ""
    sofar = []
    for i in seq:
        if(i<=15999):
            sofar.append(i)
        else:
            outp += tok.decode(sofar)
            outp += d[i]
    return outp

In [219]:
tot = 0
num = 0
ratings = 0
correct = 0
for batch_idx, (x, y, lengths) in enumerate(icl_loader):
    x = x.to(device)
    y = y.to(device)
    lengths = lengths.to(device)
    pred_logits = model(x,lengths)
    print(x.shape)
    print(x[0])
    for sequence in x:
        print(decode_seq(sequence.tolist()))
        print()
    print("========================")
    pred_tokens = torch.argmax(pred_logits,dim=2)
    pred_tokens = pred_tokens.to('cpu')
    pred_last_token = pred_tokens[:,-1].tolist()
    print(pred_tokens.shape)
    for sequence in pred_tokens:
        print(decode_seq(sequence.tolist()))
        print()
    pred_scores = [tok.decode([token]) for token in pred_last_token]
    print(pred_scores)
    print(y.tolist())
    tot += len(y)
    for t in range(len(y)):
        correct += 1 if pred_scores[t] == str(y[t]) else 0
    isratings, isnums = check_token_list(pred_scores)
    num += isnums
    ratings += isratings
    torch.cuda.empty_cache()
    break

torch.Size([8, 191])
tensor([16000,  7421,   292,  1955,    13,  1576,   174,     3,  1602,   288,
           31,   208,   165, 16002, 13254,   292,  1736, 10110, 16001, 16000,
         7421,   292,    83,   163,    75,     7,  2550,     9,    46,   106,
           51,   492,     6,     5,    54,     4,   398,    94,   701,    64,
            9,   127,     8,    54,    17,  2836,     3,     4,    88,  1015,
         1192,     7,     4,   102,   137,     8,   956,    17,  1015,   586,
           79,     3,  1831,    51,     3,  1185,   203,    33,   116,   897,
          248,   409,     6,   129,    69,  9276,  1372,    17,    31,   732,
            3, 16002, 13254,   292,  1736,  2301,  1974, 16001, 16000,  7421,
          292,  3267,   973,  6245,  8655,  1535,  6757,  1200,  2609,   935,
         1547, 11506,   808,  3317, 13971,  1073, 16002, 13254,   292,  1736,
        14419, 16001, 16000,  7421,   292,   168,    40,   354,     6,    36,
           40,    49,    11,   482,     6, 

In [150]:
print("tot: ", tot)
print("num: ", num)
print("ratings: ", ratings)
print("correct: ", correct)

tot:  8
num:  1
ratings:  1
correct:  0
